In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
import json
import datetime
import shutil
import openai
import pandas as pd
from datasets import load_dataset
from urllib.parse import parse_qs, urlparse
import base64
import time


In [2]:
from datasets import load_dataset
dataset = load_dataset("PatronusAI/financebench")

In [3]:
df = pd.DataFrame(dataset['train'])

In [13]:
df

,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number
0,financebench_id_03029,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,What is the FY2018 capital expenditure amount ...,$1577.00,Table of Contents \n3M Company and Subsidiarie...,60
1,financebench_id_04672,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,Assume that you are a public equities analyst....,$8.70,Table of Contents \n3M Company and Subsidiarie...,58
2,financebench_id_00499,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,Is 3M a capital-intensive business based on FY...,"No, the company is managing its CAPEX and Fixe...",3M Company and Subsidiaries\n Consolidated Sta...,"48,50,52"
3,financebench_id_01226,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,What drove operating margin change as of FY202...,Operating Margin for 3M in FY2022 has decrease...,"SG&A, measured as a percent of sales, increase...",27
4,financebench_id_01865,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,novel-generated,"If we exclude the impact of M&A, which segment...",The consumer segment shrunk by 0.9% organically.,Worldwide Sales Change\nBy Business Segment Or...,25
...,...,...,...,...,...,...,...,...,...
145,financebench_id_00215,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Is Verizon a capital intensive business based ...,Yes. Verizon's capital intensity ratio was app...,Consolidated Balance Sheets \nVerizon Communic...,"56, 23"
146,financebench_id_00566,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Has Verizon increased its debt on balance shee...,No. Verizon's debt decreased by $229 million.,"At December 31, Maturities \nInterest \nRates ...",77
147,financebench_id_06247,WALMART_2018_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2018,metrics-generated,What is FY2018 days payable outstanding (DPO) ...,42.69,Walmart Inc.\nConsolidated Statements of Incom...,"57,59"
148,financebench_id_04784,WALMART_2019_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2019,metrics-generated,Based on the information provided primarily in...,0.2%,Walmart�Inc.\nConsolidated�Statements�of�Incom...,48


In [4]:
filtered_df = df[df['doc_name'] == 'AMD_2015_10K']


In [5]:
filtered_df['answer']

30    4.2%
Name: answer, dtype: object

In [6]:
print(filtered_df['question'].to_list())

['Answer the following question as if you are an equity research analyst and have lost internet connection so you do not have access to financial metric providers. According to the details clearly outlined within the P&L statement and the statement of cash flows, what is the FY2015 depreciation and amortization (D&A from cash flow statement) % margin for AMD?']


- __non_pdf_files__ and __pdf_files__

In [8]:
pdf_files_urls = []
not_pdf_files_urls = []

In [9]:
pdf_files_urls

[]

In [10]:
for link in df['doc_link'].dropna().unique():
    if link.endswith('.pdf'):
        pdf_files_urls.append(link)
    else:
        not_pdf_files_urls.append(link)

- Try lock is code attempt to download
- If download due to err then except trigger

# Downloading not_pdf_files_url and download it as pdf

In [12]:
import requests

def download_pdf(url, save_path):
    """
    Download a PDF from a given URL and save it to a specified path.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")




- Manually download pdf for the 10Q or 10K for the fail download

In [13]:
for url in not_pdf_files_urls:
    file_name = url.split("/")[-1]  # Extract the filename from the URL
    file_name = file_name + '.pdf'
    save_path = f"financial_statements/{file_name}"  # Specify where to save the PDF
    download_pdf(url, save_path)

Downloaded: financial_statements/32abe798-add2-4770-9c7d-4cd3a840ede2.pdf
Error downloading https://www.adobe.com/pdf-page.html?pdfTarget=aHR0cHM6Ly93d3cuYWRvYmUuY29tL2NvbnRlbnQvZGFtL2NjL2VuL2ludmVzdG9yLXJlbGF0aW9ucy9wZGZzL0FEQkUtMTBLLUZZMTUtRklOQUwucGRm: [Errno 22] Invalid argument: 'financial_statements/pdf-page.html?pdfTarget=aHR0cHM6Ly93d3cuYWRvYmUuY29tL2NvbnRlbnQvZGFtL2NjL2VuL2ludmVzdG9yLXJlbGF0aW9ucy9wZGZzL0FEQkUtMTBLLUZZMTUtRklOQUwucGRm.pdf'
Error downloading https://www.adobe.com/pdf-page.html?pdfTarget=aHR0cHM6Ly93d3cuYWRvYmUuY29tL2NvbnRlbnQvZGFtL2NjL2VuL2ludmVzdG9yLXJlbGF0aW9ucy9wZGZzL0FEQkUtMTBLLUZZMTYtRklOQUwucGRm: [Errno 22] Invalid argument: 'financial_statements/pdf-page.html?pdfTarget=aHR0cHM6Ly93d3cuYWRvYmUuY29tL2NvbnRlbnQvZGFtL2NjL2VuL2ludmVzdG9yLXJlbGF0aW9ucy9wZGZzL0FEQkUtMTBLLUZZMTYtRklOQUwucGRm.pdf'
Error downloading https://www.adobe.com/pdf-page.html?pdfTarget=aHR0cHM6Ly93d3cuYWRvYmUuY29tL2NvbnRlbnQvZGFtL2NjL2VuL2ludmVzdG9yLXJlbGF0aW9ucy9wZGZzL0FEQkUtMTBLLUZZMTct

 - pdf_files_urls

In [10]:
for url in pdf_files_urls:
    file_name = url.split("/")[-1]  # Extract the filename from the URL
    save_path = os.path.join("financial_statements", file_name)  # Specify where to save the PDF
    download_pdf(url, save_path)

Downloaded: financial_statements\0001558370-19-000470.pdf
Downloaded: financial_statements\0000066740-23-000014.pdf
Downloaded: financial_statements\0000066740-23-000058.pdf
Downloaded: financial_statements\3be55db1-bf7e-407a-8062-6a6b8d9dd2f5.pdf
Downloaded: financial_statements\02da2a86-7bb9-4bcb-95ae-4ce27ea5e3bc.pdf
Downloaded: financial_statements\4d39f579-19d8-4119-b087-ee618abf82d6.pdf
Downloaded: financial_statements\Annual%20report%20%5BSection%2013%20and%2015(d)%2C%20not%20S-K%20Item%20405%5D.pdf
Downloaded: financial_statements\Current%20report.pdf
Downloaded: financial_statements\Annual%20report%20%5BSection%2013%20and%2015(d)%2C%20not%20S-K%20Item%20405%5D.pdf
Downloaded: financial_statements\Quarterly%20report%20%5BSections%2013%20or%2015(d)%5D.pdf
Downloaded: financial_statements\Amcor_FY23_-_Press_Release_FINAL.pdf
Downloaded: financial_statements\0000002488-16-000111.pdf
Downloaded: financial_statements\0000002488-23-000047.pdf
Downloaded: financial_statements\AMEX-10K